### RiskCalc examples.
Make sure that the working directory for this ipynb notebook is ./risktables/risktables

___
The class ```RiskCalcs``` performs basic portfolio risk calculations.  It returns a dictionary, whose items are either instances of Pandas ```DataFrame``` or floating point values representing portfolio aggregate risk measurements (VaR, Greeks, etc).
___

In [1]:
# include both the project package, and the project folder in sys.path
import sys,os
if  not './' in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not '../' in sys.path:
    sys.path.append(os.path.abspath('../'))

from risktables import risk_tables
import pandas as pd
from IPython import display
import datetime

/Users/bperlman1/Documents/billybyte/pyliverisk/risktables/risktables/dgrid_components.py:17: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/Users/bperlman1/Documents/billybyte/pyliverisk/risktables/risktables/dgrid_components.py:18: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/Users/bperlman1/Documents/billybyte/pyliverisk/risktables/risktables/dgrid_components.py:21: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


#### Create an instance of RiskCalcs

In [ ]:
# df_postgres_info = pd.read_csv('postgres_info.csv')
# display.display(df_postgres_info)
# prices_db_info = df_postgres_info.loc[df_postgres_info.config_name=='dashrisk_local']
rt = risk_tables.RiskCalcs(use_postgres=False)

#### Create an example portfolio consisting of hypothetical options on the S&P 500 Sector Spdr's

In [ ]:
next_year = (datetime.datetime.now() + datetime.timedelta(weeks=52)).year
next_year = str(next_year)
df_spdr_options =  pd.read_csv('spdr_stocks.csv')
df_spdr_options.symbol = df_spdr_options.symbol.apply(lambda s: s.replace('2021',next_year))
display.display(df_spdr_options)


#### Run risk calculations
*Market data must be fetched for each underlying, which causes the cell below to take about 10 seconds to run*

In [ ]:
rt = risk_tables.RiskCalcs(use_postgres=False)
risk_dictionary = rt.calculate(df_spdr_options)

#### List the names of the outputs

In [ ]:
for k in risk_dictionary.keys():
    print(k)

#### Now show the actual values of each key

In [ ]:
for k in risk_dictionary.keys():
    print(f'********************* {k} **************************')
    dict_item = risk_dictionary[k]
    object_to_print = dict_item
    if type(dict_item)==dict:
        if k[:2] == 'df':
            object_to_print = risk_tables.make_df(risk_dictionary[k])
    display.display(object_to_print)


In [ ]:
# pd.DataFrame(risk_dictionary['df_risk_all']).to_dict(orient="records")

In [ ]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

from dash import Dash,dash_table, html, dcc
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd

# external_scripts = [
#     {
#         'src': 'https://cdn.jsdelivr.net/npm/bootstrap@3.4.1/dist/js/bootstrap.min.js',
#         'integrity': 'sha384-aJ21OjlMXNL5UyIl/XNwTMqvzeRMZH2w8c5cRVpzpU8Y5bApTppSuUkhZXN0VxHd',
#         'crossorigin': 'anonymous'
#     },
# ]

# # external CSS stylesheets
# external_stylesheets = [
#     {
#         'href':"https://cdn.jsdelivr.net/npm/bootstrap@3.4.1/dist/css/bootstrap.min.css",
#         'rel': 'stylesheet',
#         'integrity': "sha384-HSMxcRTRxnN+Bdg0JdbxYKrThecOKuH5zCYotlSAcp1+c8xmyTe9GYg1l9a69psu" ,
#         'crossorigin': 'anonymous'
#     },
#     {
#         'href':"https://cdn.jsdelivr.net/npm/bootstrap@3.4.1/dist/css/bootstrap-theme.min.css",
#         'rel': 'stylesheet',
#         'integrity': "sha384-6pzBo3FDv/PJ8r2KRkGHifhEocL+1X2rVCTTkUfGk7/0pbek5mMa1upzvWbrUbOZ" ,
#         'crossorigin': 'anonymous'
#     },
    
# ]


# app = Dash(
#     __name__,
#     external_scripts=external_scripts,
#     external_stylesheets=external_stylesheets
# )
app = Dash(
    __name__,
)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
# df = pd.DataFrame({
#     "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
#     "Amount": [4, 1, 2, 2, 4, 5],
#     "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
# })

# fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

df_risk_all = risk_tables.make_df(risk_dictionary['df_risk_all'])
dt = dash_table.DataTable(
    df_risk_all.to_dict('records'), 
    [
        {"name": i, "id": i} 
        for i in df_risk_all.columns
    ],
)
layout = html.Div(
    children=[
        html.Div(
#             style="width:98vw",
#             className="containerFluid",
            children=[
                html.H1(
                    children='Hello Dash',
                ),
                html.Div(
                    children='''
                    Dash: A web application framework for your data.
                    ''',
#                     className="row"
                ),
                html.Div(
#                     className="row",
                    children=[dt],
                ),
#                 dcc.Graph(
#                     id='example-graph',
#                     figure=fig
#                 ),        
            ],
        )
    
])

app.layout = layout
app.run_server(port=8999,debug=False,)

    